In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

In [10]:
#data = pd.read_excel('./Network_LA_uc1/join_junc_to_links_uc1_locals.xlsx')
#data_pos = pd.read_excel("./Network_LA_uc1/uc1_02_05_locals_ND_Junctions_UTM.xlsx")
#--- uc3
#data = pd.read_csv('./TOTAL_FILE/join_junc_to_CA_02_05_simple_filed.csv')
#data_pos = pd.read_excel("./TOTAL_FILE/CA_02_05_simple_field_ND_Junctions_UTM.xlsx")
data = pd.read_csv('./TOTAL_FILE/join_junc_to_CA_02_05_simple_filed.csv')
data_pos = pd.read_excel("./TOTAL_FILE/CA_02_05_simple_field_ND_Junctions_UTM.xlsx")

#data = pd.read_csv('./uc6_local/join_junc_to_links_uc6_locals.csv')
#data_pos = pd.read_excel("./uc6_local/uc6_02_05_UTM_locals_ND_Junctions.xlsx")
print (data.head(10))


   LinkID    JID1    JID2             NAME
0       0  354243  354245       Highway 33
1       1  353214  353245        Rincon Rd
2       2  353461  353467        Rincon Rd
3       3  353808  353811  Casitas Pass Rd
4       4  315075  315132   Bitterwater Rd
5       5  315132  315141   Bitterwater Rd
6       6  314989  315075   Bitterwater Rd
7       7  117446  117568        Rivera Rd
8       8  117568  117582        Rivera Rd
9       9   77080   77104    Foothill Blvd


In [11]:
#build the dictionary about 
#1.The junction connects to
#2. The the edge ID is connect to which two junction
#3. Which Junction is connected to more than thre edges.
def Build_dict(df):
    multiple_road = {} # store which junction coneected to more than two roads
    dic = {} # store junction A connected to junction B
    street_id = {} # store pair of junction with the edge
    for i in range(len(df)):
        row = df.iloc[i,:]
        if row[1] not in dic:
            dic[row[1]] = [row[2]]
        elif row[2] not in dic[row[1]]:
            dic[row[1]].append(row[2])
        if row[2] not in dic:
            dic[row[2]] = [row[1]]
        elif row[1] not in dic[row[2]]:
            dic[row[2]].append(row[1])
        smaller,bigger = min(row[1],row[2]),max(row[1],row[2])
        if (smaller,bigger) in street_id:
            All_Deleted_Edge.append([row[0]])
        else:
            street_id[(smaller,bigger)] = row[0]
    for num in dic:
        if len(set(dic[num])) >=2:
            multiple_road[num] = dic[num]
    print ("end reading data")
    return multiple_road,dic,street_id

In [12]:
#Utilize the DFS to find all the circle with the main street name
#it might contains a lot of circles
#However, we can know that # unique group of circle is not as many as original one 
#seen recorded sorted edge
#seen_pt record path of pt
#seen_edge record edges of path
def DFS(path,pre_element,next_element,seen,seen_pt,seen_edge):#seen saved the seen path
    if next_element in path:# it means head to head
        #path = path+[next_element]   
        path = path[path.index(next_element):]
        cur_street = []
        for index in range(len(path)-1):
            smaller = path[index]
            bigger = path[index+1]
            if smaller > bigger:
                smaller,bigger = bigger,smaller
            cur_street.append(street_id[(smaller,bigger)])
        if sorted(cur_street) in seen: # make sure there is no redundant loop
            return None
        seen_edge.append(cur_street)
        seen.append(sorted(cur_street))
        seen_pt.append(path)
        return path 
    flag = None
    for new in dic[next_element]:
        if new not in path[1:]:# make sure the circle should be from head to head
            flag = DFS(path+[next_element],next_element,new,seen,seen_pt,seen_edge)
            if flag != None:
                flag = True
                break
    if flag == None:
        return None
    return path

In [13]:
# Because some circle should be sub-circle of the larger one
# We need to Union all of them and find out how many group of large circle
# And we only need to handle those Large Circles
def find_group_circle(seen_pt,seen_edge): # union all the circle and find out unique group of circle
    dif_group = 0
    dif_group_pt = []
    dif_group_edge = []
    for i,val in enumerate(seen_pt):
        flag = False
        for j in range(i+1,len(seen_pt)): # if any circle is concluded in the larger cricle, remove this circle
            if all(n in seen_pt[j] for n in val):
                flag = True
                break
        if flag == False:
            dif_group_pt.append(val)
            dif_group_edge.append(seen_edge[i])
            dif_group+=1
    print ("total %d different group" %(dif_group))
    return dif_group_pt,dif_group_edge

In [14]:

# read_pos:
# read the position of all the pts in this circle
# And utilize the X or Y pos to determine the circle is North-South or East-West Dir and 
# read_critical_pt:
# Based on the direction to determine which pts are boundary pt, starting pt and ending pt
# get_the_route_by_critical_pt:
# The function is to find the path from the starting pt to ending pt which contains more junction
# besides Route_pt is not sorted, it means it keep the situation how the pts coneected

def read_pos(route_pt,route_edge): # read the position for the circle and find out the direction of the circle
    X_pos = []
    Y_pos = []
    for pt in route_pt:
        print (route_pt)
        X_pos.append(data_pos.ix[pt-1,1]) # id 1 is first row
        Y_pos.append(data_pos.ix[pt-1,2])
    N_S_Flag = False
    if np.std(Y_pos) > np.std(X_pos):
        N_S_Flag = True
    return read_critical_pt(N_S_Flag,X_pos,Y_pos,route_pt,route_edge)
def read_critical_pt(N_S_Flag,X_pos,Y_pos,route_pt,route_edge): #Use the direction to get the critical pt pair <northest,southest> or <eastest,westesst>
    critical_pt1= critical_pt2 = -1 # store the boundary pt
    if N_S_Flag == True:
        critical_pt1 = Y_pos.index(max(Y_pos)) # find the northest pt index
        critical_pt1 = route_pt[critical_pt1] # get the id of the point
        critical_pt2 = Y_pos.index(min(Y_pos)) # find the southest pt index
        critical_pt2 = route_pt[critical_pt2] # get the id of the point
    else: # east west dir , haven't test it 
        critical_pt1 = X_pos.index(max(X_pos)) # find the eastest pt index
        critical_pt1 = route_pt[critical_pt1] # get the id of the point
        critical_pt2 = X_pos.index(min(X_pos)) # find the westest pt index
        critical_pt2 = route_pt[critical_pt2] # get the id of the point
    return get_the_route_by_critical_pt(critical_pt1,critical_pt2,route_pt,route_edge)
def get_the_route_by_critical_pt(critical_pt1,critical_pt2,route_pt,route_edge):
    #print ("------------start_get_route---------------")
    #print (route_pt)
    start_index = route_pt.index(critical_pt1)
    end_index = route_pt.index(critical_pt2)
    #print ("path index = ",start_index,end_index)
    start_index,end_index = min(start_index,end_index),max(start_index,end_index)
    if end_index - start_index+1 > int(len(route_pt)/2):
        #print ("From start to end")
        route_pt_saved =route_pt[start_index:end_index+1]
        route_edge_saved= route_edge[start_index:end_index]
    else:
        route_pt_saved =route_pt[end_index:] + route_pt[:start_index+1]
        route_edge_saved= route_edge[end_index:] + route_edge[:start_index]
    get_del_edge(route_pt_saved,route_pt,route_edge_saved,route_edge)
    return route_edge_saved,route_pt_saved
def get_del_edge(route_pt_saved,route_pt,route_edge_saved,route_edge):
    #return 0 
    del_edges= []
    del_pts = [v for v in route_pt if v not in route_pt_saved]
    del_pts = sorted(del_pts)
    for i,pts in enumerate(del_pts):
        for val in dic[pts]:
            s_pt,l_pt = min(val,pts),max(val,pts)
            if (s_pt,l_pt) in street_id:
                #for edge in street_id[(s_pt,l_pt)]:
                del_edges.append(street_id[(s_pt,l_pt)])
                #del_edges.append(edge)
    for val in route_edge:
        if val not in route_edge_saved and val not in del_edges:
            del_edges.append(val)
    
    if len(del_edges) !=0:
        print ("delete_edges = ",del_edges)
        All_Deleted_Edge.append(del_edges)


In [15]:
def build_main_street_dic():
    for index,(LinkID,JID1,JID2,StName) in enumerate(zip(data['LinkID'],data['JID1'],data['JID2'],data['NAME'])):
        Main_StName = str(StName).split(" ")
        if Main_StName[0] in Direction:
            Main_StName = Main_StName[1:]
        if Main_StName[-1] in Direction:
            Main_StName = Main_StName[:-1]
        Main_StName= ''.join(Main_StName)
        Street_Name[Main_StName]= Street_Name.get(Main_StName,[])
        Street_Name[Main_StName].append(LinkID)


In [ ]:

All_Deleted_Edge = []
#data = pd.read_excel('./Network_LA/uc6_locals_junc_pair_v1.xlsx')
Street_Name = {} # record which index is store in the name of streer
Direction = ['E','N','S','W']
build_main_street_dic() # call func
#print ("streets #:",Street_Name["PioneerBlvd"])
now = datetime.now()
for index,name in enumerate(Street_Name):
    #"PioneerBlvd" "HayvenhurstAve"
    df = data.loc[data['LinkID'].isin(Street_Name[name])] # find the all rows with same street name
    multiple_road = {} # store which junction coneected to more than two roads
    dic = {} # store junction A connected to junction B
    street_id = {} # store pair of junction with the edge
    seen = [] # record the route edge
    seen_edge = []
    seen_pt = [] # record the  route point
    multiple_road,dic,street_id = Build_dict(df)
    print ("check pt one")
    #print (multiple_road)
    for start_pos in multiple_road:
        for i in multiple_road[start_pos]:
            if DFS([start_pos],start_pos,i,seen,seen_pt,seen_edge) != None:
                break
    print ("check pt two")
    print ("total %d groups"%(len(seen)))
    seen = sorted(seen,key = lambda x:len(x))
    seen_pt = sorted(seen_pt,key = lambda x:len(x))
    seen_edge = sorted(seen_edge,key = lambda x:len(x))
    #filename = "./Network_LA/uc6_02_05_UTM_locals_ND_Junctions.xlsx"
    dif_group_pt,dif_group_edge = find_group_circle(seen_pt,seen_edge)
    for route_pt,route_edge in zip(dif_group_pt,dif_group_edge):
        read_pos(route_pt,route_edge)
later = datetime.now()
difference = (later - now).total_seconds()
print ("total time for go through a steet is ",difference)

end reading data
check pt one
check pt two
total 439 groups
total 437 different group
[354243, 354245]
[354243, 354245]
[353229, 353253]
[353229, 353253]
[353253, 353254]
[353253, 353254]
[353710, 353732]
[353710, 353732]
[353732, 353750]
[353732, 353750]
[353792, 353814]
[353792, 353814]
[353814, 353915]
[353814, 353915]
[353268, 353297]
[353268, 353297]
[353297, 353299]
[353297, 353299]
[352777, 352801]
[352777, 352801]
[352801, 352803]
[352801, 352803]
[354185, 354241]
[354185, 354241]
[354241, 354243]
[354241, 354243]
[353490, 353512]
[353490, 353512]
[353512, 353545]
[353512, 353545]
[353347, 353417]
[353347, 353417]
[353417, 353429]
[353417, 353429]
[353429, 353483]
[353429, 353483]
[353483, 353488]
[353483, 353488]
[353167, 353198]
[353167, 353198]
[353198, 353209]
[353198, 353209]
[353314, 353324]
[353314, 353324]
[353324, 353347]
[353324, 353347]
[352622, 352651]
[352622, 352651]
[352651, 352655]
[352651, 352655]
[352954, 353018]
[352954, 353018]
[353018, 353148]
[353018, 3531

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  app.launch_new_instance()
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


[348890, 348928]
[348890, 348928]
[339425, 339450]
[339425, 339450]
[339450, 339479]
[339450, 339479]
[354199, 354358]
[354199, 354358]
[354358, 354364]
[354358, 354364]
[322245, 322478]
[322245, 322478]
[322478, 322761]
[322478, 322761]
[354941, 354964]
[354941, 354964]
[354964, 354966]
[354964, 354966]
[349833, 349858]
[349833, 349858]
[349858, 349886]
[349858, 349886]
[350395, 350401]
[350395, 350401]
[350401, 350413]
[350401, 350413]
[350695, 350731]
[350695, 350731]
[350731, 350762]
[350731, 350762]
[349963, 350008]
[349963, 350008]
[350008, 350015]
[350008, 350015]
[350302, 350378]
[350302, 350378]
[350378, 350390]
[350378, 350390]
[354727, 354736]
[354727, 354736]
[354736, 354759]
[354736, 354759]
[350413, 350443]
[350413, 350443]
[350443, 350524]
[350443, 350524]
[349951, 349959]
[349951, 349959]
[349886, 349925]
[349886, 349925]
[334065, 334352]
[334065, 334352]
[334352, 335004]
[334352, 335004]
[351712, 351798]
[351712, 351798]
[351798, 351835]
[351798, 351835]
[354966, 35497

[354010, 354083]
[354010, 354083]
[343554, 343636]
[343554, 343636]
[252409, 253468]
[252409, 253468]
[253468, 255694]
[253468, 255694]
[261742, 261376]
[261742, 261376]
[235261, 235511]
[235261, 235511]
[262322, 262261]
[262322, 262261]
[247977, 247724]
[247977, 247724]
[248251, 249356]
[248251, 249356]
[249356, 252340]
[249356, 252340]
[255694, 256610]
[255694, 256610]
[262415, 262322]
[262415, 262322]
[314321, 314216]
[314321, 314216]
[252355, 252369]
[252355, 252369]
[252369, 252409]
[252369, 252409]
[248212, 247977]
[248212, 247977]
[248238, 248212]
[248238, 248212]
[252340, 252355]
[252340, 252355]
[262591, 262583]
[262591, 262583]
[314645, 314745]
[314645, 314745]
[352340, 352312, 352356]
[352340, 352312, 352356]
[352340, 352312, 352356]
delete_edges =  [16466, 176614, 198437]
end reading data
check pt one
check pt two
total 23 groups
total 23 different group
[353214, 353245]
[353214, 353245]
[353245, 353248]
[353245, 353248]
[353461, 353467]
[353461, 353467]
[353467, 353471]
[3

[315353, 315429]
[315606, 315654]
[315606, 315654]
[315654, 315817]
[315654, 315817]
[317517, 317543]
[317517, 317543]
[268393, 268418]
[268393, 268418]
[316966, 316977]
[316966, 316977]
[316977, 316983]
[316977, 316983]
[273207, 273590]
[273207, 273590]
[273590, 273707]
[273590, 273707]
[272939, 273207]
[272939, 273207]
[272210, 272939]
[272210, 272939]
[317543, 317554]
[317543, 317554]
[317514, 317523]
[317514, 317523]
[317523, 317515]
[317523, 317515]
[269110, 269126]
[269110, 269126]
[269126, 269179]
[269126, 269179]
[269498, 269861]
[269498, 269861]
[269861, 269893]
[269861, 269893]
[273707, 273720]
[273707, 273720]
[317524, 317525]
[317524, 317525]
[273766, 273828]
[273766, 273828]
[317564, 317654]
[317564, 317654]
[317654, 318046]
[317654, 318046]
[318063, 318330]
[318063, 318330]
[318330, 318416]
[318330, 318416]
[318060, 317765]
[318060, 317765]
[269179, 269498]
[269179, 269498]
[268024, 268038]
[268024, 268038]
[268447, 268464]
[268447, 268464]
[268464, 268472]
[268464, 26847

In [ ]:
data_tocsv = []
for row in All_Deleted_Edge:
    #print (row)
    for element in row:
        data_tocsv.append(element)
data_tocsv = sorted(set(data_tocsv))
print (len(set(data_tocsv)))
df = pd.DataFrame(data_tocsv, columns=["LinkID"])
df.to_csv('./TOTAL_FILE/delete_local.csv', index=False)

In [27]:
# 2089 for not delete only two junctions old version

In [ ]:
# This is for multiple districts in dif data
#!/usr/bin/python
# -*- coding: utf-8 -*-
'''
from os import listdir
from os.path import isfile, isdir, join

mypath = "C:/Users/user/Documents/ArcGIS"
files = listdir(mypath)
folders = []
file = []
for f in files:
    fullpath = join(mypath, f)
    if isfile(fullpath):
        print("file：", f)
        file.append(f)
    elif isdir(fullpath):
        print("Folder：", f)
        folders.append(f)
'''